In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.axes import Axes

from Global_Parameter import GlobalParams
from Green_Function import GKTH_Greens
from H_eig import GKTH_find_spectrum
from k_space_flipping import GKTH_flipflip
from Layer import Layer
from self_consistency_delta import GKTH_self_consistency_1S_find_root, GKTH_self_consistency_1S_iterate

# Band energy spectrums

In [ ]:
# Default parameters and single default layer
p = GlobalParams()
layers = [Layer()]

max_val_kx = np.max(np.abs(p.k1))
kx = np.linspace(-max_val_kx, max_val_kx, p.nkpoints * 2)
ky = np.linspace(-max_val_kx, max_val_kx, p.nkpoints * 2)
kx, ky = np.meshgrid(kx, ky)

# energy is 3d array, nkpoints x nkpoints x (4*nlayers)
# each k-point has its only eigenvalues list
energy = GKTH_find_spectrum(p, layers)

bands = []
for i in range(4 * len(layers)):
    energy_band = energy[:, :, i]
    new_band = GKTH_flipflip(energy_band)
    bands.append(new_band)

In [ ]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

for i, band in enumerate(bands):
    # Skip bands with NaNs or infinities
    if np.isnan(band).any() or np.isinf(band).any():
        print(f"Skipping band {i} due to NaN or infinity values.")
        continue

    surf = ax.plot_surface(kx, ky, band, facecolor=f"C{i%10}", alpha=0.5)

ax.set_xlabel("$k_x$")
ax.set_ylabel("$k_y$")
ax.set_zlabel("Energy (eV)")
ax.set_title("Energy Spectrum")

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

for i, band in enumerate(bands):
    ax: Axes = axes[i // 2, i % 2]
    ax.imshow(band, cmap="viridis")
    ax.set_title(f"Band {i}")
    ax.set_xlabel("$k_x$")
    ax.set_ylabel("$k_y$")

fig.tight_layout()

# Residual-Gap Plot

## Check root finding

In [ ]:
h_list = [0.0, 7.5e-4, 8e-4, 1e-3]
plot_tuples = []
Deltas = []

for h in h_list:
    p = GlobalParams(h=h)

    # Find root where residuals are zero
    layers = [Layer()]
    Delta, layers = GKTH_self_consistency_1S_find_root(p, layers)
    Deltas.append(Delta)

    # See how residuals vary with Delta to check find root
    layers = [Layer()]
    x_vals, residuals = GKTH_self_consistency_1S_iterate(p, layers)
    plot_tuples.append((x_vals, residuals))

In [ ]:
plt.figure(figsize=(12, 6))

for i, (h, (x_vals, residuals), Delta) in enumerate(zip(h_list, plot_tuples, Deltas)):
    plt.plot(x_vals, residuals, label=f"h = {h}", color=f"C{i}")
    plt.scatter([Delta], [0], color=f"C{i}", marker="x")

plt.axhline(y=0, color="gray", linestyle="--")
plt.xlabel("Delta_0 (eV)")
plt.ylabel("Residual (eV)")
plt.title("Residual vs Delta_0")
plt.legend()
plt.show()

## Gap vs h

In [ ]:
h_list = np.linspace(0, 1e-3, 20)

for h in h_list:
    p = GlobalParams(h=h)
    layers = [Layer()]

    Delta, layers = GKTH_self_consistency_1S_find_root(p, layers)
    Deltas.append(Delta)

In [ ]:
plt.plot(h_list, Deltas[4:], linestyle="--", marker="o", label=r"$\lambda = 0.1$")
plt.xlabel("h (eV)")
plt.ylabel("Delta (eV)")
plt.legend()

# Matsubara Frequencies

In [ ]:
%matplotlib inline
matsubara_freqs, ksums, F_kresolved_final = GKTH_Greens(p, layers, verbose = True)

In [ ]:
plt.pcolormesh(np.real(F_kresolved_final[0, ...]))
plt.colorbar()
plt.title("Real part of F_kresolved_final")
plt.show()

plt.scatter(matsubara_freqs, np.abs(ksums))
plt.xlim(0, 5200)
plt.title("Matsubara Frequencies vs |ksums|")
plt.xlabel("Matsubara Frequencies")
plt.ylabel("|ksums|")
plt.show()

# Dump

In [11]:
# fig = plt.figure()
# ts_list = []
# for i in range(10):
#     ts = 0.9 + i/50
#     p.ts = np.zeros(100) + ts
#     delta_list = []
#     tNN_list = []
#     for j in range(10):
#         print("current iteration:", i, j)
#         ts_list.append(ts)
#         tNN = -1 - j/20
#         Nb.tNN = tNN
#         Nb.tNNN = Nb.tNN * 0.1
#         tNN_list.append(tNN)
#         delta,_, _ = GKTH_self_consistency_1S(p,layers)
#         delta_list.append(delta)
#     plt.plot(tNN_list, delta_list, label = f"{ts:.2f}")
    
# plt.xlabel("tNN (eV)")
# plt.ylabel("delta_best_fit (eV)")
# plt.legend()